In [1]:
# @title basic functions
def R(i=None): #
  x=list(str(i))

  for j in [0,1,4,5]:
    if int(x[j])+5 >12:
      continue
    elif (j+int(x[j]))%2:
      x[int(x[j])+5]=str((int(x[int(x[j])+5])+1)%3)
    else:
      x[int(x[j])+5]=str((int(x[int(x[j])+5])-1)%3)

  x[0],x[4]=x[4],x[0]
  x[0],x[1]=x[1],x[0]
  x[4],x[5]=x[5],x[4]

  return int("".join(x))

def R_(i=None): #
  x=list(str(i))

  for j in [0,1,4,5]:
    if int(x[j])+5 >12:
      continue
    elif (j+int(x[j]))%2:
      x[int(x[j])+5]=str((int(x[int(x[j])+5])+1)%3)
    else:
      x[int(x[j])+5]=str((int(x[int(x[j])+5])-1)%3)

  x[1],x[5]=x[5],x[1]
  x[0],x[1]=x[1],x[0]
  x[4],x[5]=x[5],x[4]

  return int("".join(x))

def B(i=None):
  x=list(str(i))

  x[6],x[3]=x[3],x[6]
  x[6],x[4]=x[4],x[6]
  x[6],x[5]=x[5],x[6]

  return int("".join(x))

def B_(i=None):
  x=list(str(i))

  x[6],x[3]=x[3],x[6]
  x[4],x[3]=x[3],x[4]
  x[5],x[4]=x[4],x[5]

  return int("".join(x))

def D(i=None):
  x=list(str(i))

  for j in [1,2,3,4]:
    if int(x[j])+5 >12:
      continue
    elif (j+int(x[j]))%2:
      x[int(x[j])+5]=str((int(x[int(x[j])+5])-1)%3)
    else:
      x[int(x[j])+5]=str((int(x[int(x[j])+5])+1)%3)

  x[1],x[2]=x[2],x[1]
  x[2],x[3]=x[3],x[2]
  x[3],x[4]=x[4],x[3]

  return int("".join(x))

def D_(i=None):
  x=list(str(i))

  for j in [1,2,3,4]:
    if int(x[j])+5 >12:
      continue
    elif (j+int(x[j]))%2:
      x[int(x[j])+5]=str((int(x[int(x[j])+5])-1)%3)
    else:
      x[int(x[j])+5]=str((int(x[int(x[j])+5])+1)%3)

  x[1],x[4]=x[4],x[1]
  x[2],x[3]=x[3],x[2]
  x[4],x[2]=x[2],x[4]

  return int("".join(x))

#for f in [R,R_,B,B_,D,D_]:
  #print(f(23456780000000))

In [5]:
# @title Generating the map (~2min)
themapkey=[]
themap=dict()
for i in itr.permutations("2345678"):
  for j in itr.product('012', repeat=6):
    themapkey+=[int("".join(i) + "".join(j))]

for i in themapkey:
  themap[i]={(f[0]+1,f[1](i)) for f in enumerate([R,R_,B,B_,D,D_])}

# a=[]
# for i in themap:
#   a+=[[i, themap[i]]]
# import csv

# filename = "/content/drive/MyDrive/Colab Notebooks/misc/cubemap.csv"

# with open(filename, 'w') as csvfile:
#     csvwriter = csv.writer(csvfile)
#     csvwriter.writerows(a)

A state of 2x2 cube is represented by a 13 digits integer: 2345678-012012, 2-8 for block index and 6 cube orientation **order by index**.
Using matrix coordinate, (0,0,1) is the first element of the backside. Ordering
is (0,0,0) > (0,1,0) > (1,1,0) > (1,0,0) >> (1,0,1) > (1,1,1) > (0,1,1) > (0,0,1)

Each block has 3 orientations at each corner. Using xyz coordinate, the state (0, 1, or 2) of a orientation represents the plane orthogonal to the axises (x,y,z) in the solved cube state. 0 where the plane facing x axis is the x plane at solved state, 1 where it's x plane is the y plane at solved state, etc.

Block is either odd or even depend on its position at solved state. (1,2,3,....)
Each move, a block goes from odd positions to even position. With the 1) parity of position at solved state(or index of the block), 2) the parity of current position, 3) Axis of rotation of the move. Three variables can determine the resulting orientation after a move.

In [12]:
#import sys
#a=23456780000000
#print("The size of the empty list is:",sys.getsizeof(  themap  ), "bytes.")

#start_time = time.time()

#s=[]
#for i in range(100000):
#  d=np.random.randint(10**13,88888888888888)
#  s+=[R(d),R(R(d)),R(R(R(d))) ]

#print(time.time() - start_time)
#3.337843656539917 for n=100000

import numpy as np
import matplotlib.pyplot as plt
import time
import itertools as itr
import csv

themapkey=[]
themap=dict()

filename = "/content/drive/MyDrive/Colab Notebooks/misc/cubemap.csv"

with open(filename, mode ='r') as file:
  csvFile = csv.reader(file)
  for lines in csvFile:
    themapkey+=[int(lines[0])]
    themap[int(lines[0])]=lines[1]

print(themapkey[0],themap[themapkey[0]])

2345678000000 {(6, 2634578011110), (2, 7245368220022), (4, 2346785000000), (1, 3645728220022), (3, 2348567000000), (5, 2456378011110)}


Color Order: (x+,x-,y+,y-,z+,z-) : "Blue", "Green", "Orange", "Red", "White", "Yellow". So first block is fixed at ("Blue", "Red", "White"), second block initial orientation is ("Blue", "Orange", "White") etc......

Steps to solve:

1. **("Blue", "Red", "White")** at top left corner.
2. Get the 13 digit state number using key. Look for color of the face at x-axis
3. Plug and follow steps.

In [24]:
colors=["Blue", "Green", "Orange", "Red", "White", "Yellow"]
blocks=['034','024','025','035','135','125','124','134']
print("COLOR KEY")
for i in range(len(blocks)):
  print(i+1,[colors[int(j)] for j in list(blocks[i])])

COLOR KEY
1 ['Blue', 'Red', 'White']
2 ['Blue', 'Orange', 'White']
3 ['Blue', 'Orange', 'Yellow']
4 ['Blue', 'Red', 'Yellow']
5 ['Green', 'Red', 'Yellow']
6 ['Green', 'Orange', 'Yellow']
7 ['Green', 'Orange', 'White']
8 ['Green', 'Red', 'White']


In [27]:
#########################################################################

state=(0, 3245678000000)  # 4 moves testcase: 4586237002020,          11 moves testcase: 8725463000012

#########################################################################

que1=[(0, 2345678000000)]
que2=[state]
nextque1=[]
nextque2=[]
s=0
d1=dict(zip(themapkey,[0]*3674160))
d2=dict(zip(themapkey,[0]*3674160))
part1,part2=0,0
while que1 and que2:
  for k in que1:
    if d2[k[1]]:
      part2,part1=k[0],d2[k[1]]
      #print(part1,part2)
      break
    else:
      d1[k[1]]=k[0]
      for f in eval(themap[k[1]]):
        if d1[f[1]]:
          continue
        else:
          nextque1+=[( 10*k[0]+f[0], f[1] )]

  for j in que2:
    if d1[j[1]]:
      part1,part2=j[0],d1[j[1]]
      #print(part1,part2)
      break
    else:
      d2[j[1]]=j[0]
      for f in eval(themap[j[1]]):
        if d2[f[1]]:
          continue
        else:
          nextque2+=[( 10*j[0]+f[0], f[1] )]
  s+=1
  if s > 20 or (part1 and part2) :
    break
  else:
    que1=nextque1
    nextque1=[]
    que2=nextque2
    nextque2=[]
reverse=lambda x: "".join([ str( ((int(i)-1)//2)*2+int(i)%2+1 ) for i in list(x)[::-1]])
solution=str(part1)+reverse(str(part2))

print(len(solution), "steps")
f=['R','R_','B','B_','D','D_']
state1=state[1]
print("Start:",state1)
for i in list(solution):
  state1=eval(f[int(i)-1])(state1)
  print(f[int(i)-1], state1 )


13 steps
Start: 3245678000000
B 3248567000000
D 3485267201200
R_ 6385427112210
D 6854327120010
R 8354267200000
R 3254687120010
D_ 3625487022220
B 3627548022220
R 6527438000000
D 6274538101101
D 6745238000000
R_ 3645728220022
R_ 2345678000000


#Diameter??


In [56]:
state=8725463000012  # 4 moves testcase: 4586237002020,          11 moves testcase: 8725463000012

#########################################################################

que1=[2345678000000]
que2=[state]
nextque1=[]
nextque2=[]
s=0
d=dict(zip(themapkey,[0]*3674160))
while que1 and que2:
  for k in que1:
    if d[k]:
      continue
    else:
      d[k]=1
      for f in eval(themap[k]):
        if d[f[1]]:
          continue
        else:
          nextque1+=[f[1]]

  for j in que2:
    if d[j]:
      continue
    else:
      d[j]=1
      for f in eval(themap[j]):
        if d[f[1]]:
          continue
        else:
          nextque2+=[f[1]]
  s+=1
  print(s)
  if s > 20 :
    break
  else:
    que1=nextque1
    nextque1=[]
    que2=nextque2
    nextque2=[]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
